In [1]:
import os, gc
from tqdm import tqdm

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma, FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain.chains import RetrievalQA

import torch

import warnings
warnings.filterwarnings('ignore')

print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

1
NVIDIA GeForce RTX 3080 Ti


In [3]:
loader = PyPDFLoader("pdf_samples/1716891429607.pdf")

docs = loader.load()

r_splitter = RecursiveCharacterTextSplitter(chunk_size = 800, chunk_overlap = 200)
docs_split = r_splitter.split_documents(docs)

In [4]:
embedding = HuggingFaceEmbeddings(
    #model_name = "tugstugi/bert-base-mongolian-cased",
    model_name = "sentence-transformers/LaBSE",
    #model_name = "bayartsogt/albert-mongolian",
    model_kwargs = {"device": "cuda"}, 
    encode_kwargs = {"normalize_embeddings": True}
)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [5]:
vector_store = Chroma.from_documents(documents = docs_split, 
                                     embedding = embedding)

faiss_index = FAISS.from_documents(documents = docs_split, 
                                   embedding = embedding)

In [6]:
def get_docs(prompt, vector_db = 'chroma', k = 10):
    if vector_db == 'chroma':
        return vector_store.similarity_search(prompt, k)
    return faiss_index.similarity_search(prompt, k)

def get_and_print_docs(prompt, vector_db, k):
    docs = get_docs(prompt, vector_db, k)
    print(f"Prompt: {prompt}")
    print(f"Vector DB: {vector_db}\n")

    for idx, doc in enumerate(docs):
        print(f"Returned: {idx + 1}")
        print(doc.page_content, '\n')

    return docs

In [7]:
prompt = 'санхүүгийн чадавхийн шаардлагад ямар баримт бичиг хамаарах вэ'
docs = get_and_print_docs(prompt, vector_db = 'chroma', k = 10)

Prompt: санхүүгийн чадавхийн шаардлагад ямар баримт бичиг хамаарах вэ
Vector DB: chroma

Returned: 1
шаардлага19.1.Хуулийн  17.4-т  зааснаар  оролцогч  өгөгдлийн  
хүснэгтэд  заасан  техникийн  чадавх  болон  туршлагын  
шаардлагыг хангасан байна.
19.2.Техникийн  болон  туршлагын  чадавхын  
шаардлагыг хангах нотлох баримт ирүүлэхийг шаардах  
бол  хуулийн  17.2-т  заасан  баримт  бичгээс  сонгож  
өгөгдлийн хүснэгтэд заана.
19.3.Бараа  нийлүүлэх,  дагалдах  үйлчилгээ  үзүүлэхэд  
шаардлагатай  зөвшөөрөл,  эсхүл  гэрчилгээ  байгаа  бол  
өгөгдлийн хүснэгтэд заана. 

Returned: 2
18, 19-д чадавхын болон туршлагын шаардлагын 40-аас  
дээш хувийг, түншлэлийн гишүүн тус бүр 25 ба түүнээс  
дээш хувийг хангасан байна.
18.Санхүүгийн 
чадавхын шаардлага18.1.Хуулийн  16.5-д  зааснаар  оролцогч  
өгөгдлийн  хүснэгтэд  заасан  санхүүгийн  чадавхын  
шаардлага хангасан байна.
18.2.Санхүүгийн  чадавхын  шаардлагыг  хангах  нотлох  
баримт  ирүүлэхийг  шаардах  бол  хуулийн  16.2,  16.4-т  
заасан б

In [8]:
# Much better???
print(docs[3].page_content)

2.1- Байгууллагын дансны үлдэгдэл /Харилцагч банкны/
2.2-  Арилжааны  банкны  зээлийн  авч  болох  мөнгөн  дүн  бүхий  
зээлийн  Тодорхойлолт  (Банкны  судлан  үзэж  зээл  олгох  
боломжтой гэсэн мэдэгдлийг үнэлгээнд харгалзахгүй). 
2.3-Үнэт  цаас,  Засгийн  газрын  бонд  болон  Үнэт  цаасны  зах  
зээлийн тухай хуулийн 5 дугаар зүйлийн 5.1 дэх хэсэгт заасан  
үнэт цааснууд. 
2.4-  Түргэн  хөрвөх  чадвартай  хөрөнгө  болон  авах  боломжтой  
зээлийн  нийт  дүнг  Сангийн  сайдын  2015  оны  343  дугаар  
тушаалаар батлагдсан “Санхүүгийн тайланд шинжилгээ хийх  
үлгэрчилсэн загвар”-ын 3.2.2 /е/-д “Түргэн хөрвөх чадвартай  
хөрөнгө=Мөнгө  Түүнтэй  Адилтгах  Хөрөнгө  +  Санхүүгийн  
хөрөнгө + Дансны авлага гэсэн үзүүлэлтүүдээр дүгнэнэ” гэж  
заасны дагуу тодорхойлно


In [9]:
print(docs[4].page_content)

хөрөнгө=Мөнгө  Түүнтэй  Адилтгах  Хөрөнгө  +  Санхүүгийн  
хөрөнгө + Дансны авлага гэсэн үзүүлэлтүүдээр дүгнэнэ” гэж  
заасны дагуу тодорхойлно
ТШЗ 18.2Санхүүгийн  чадавхыг  хянан  үзэхэд  үндэслэл  болгох  баримт  
бичиг:Шаардана
1.Зээлийн  мэдээллийн  тухай  хуулийн  4.1.5-д  заасан  зээлийн  
мэдээллийн  үйлчилгээ  үзүүлэх  этгээдээс  тендер  шалгаруулалт  
зарласан өдрөөс хойш авсан, шалгах боломжтой QR код бүхий  
“Зээлийн  мэдээллийн  сангийн  тодорхойлолт”-ыг  ирүүлнэ.  Уг  
тодорхойлолтод төлөгдөж дуусаагүй зээлийн мэдээлэл “Хэвийн”  
бус  байгаа  тохиолдолд  татгалзах  үндэслэл  болно.  (“Зээлийн  
мэдээллийн  сангийн  тодорхойлолт”-ыг  www.sainscore.mn,  
www.burenscore.mn  хаягаар  хандан  авна.  Харилцагч  банкны  
тодорхойлолтыг тендерийг үнэлэхэд харгалзан үзэхгүй.)


In [10]:
prompt = 'түргэн хөрвөх хөрөнгөтэй холбогдох ямар шаардлага байгаа вэ'
docs = get_and_print_docs(prompt, vector_db = 'chroma', k = 10)

Prompt: түргэн хөрвөх хөрөнгөтэй холбогдох ямар шаардлага байгаа вэ
Vector DB: chroma

Returned: 1
бэлтгэхэд оролцогчдод зайлшгүй чухал нөхцөл болно. 
Техникийн  тодорхойлолтыг  бэлтгэхэд  өрсөлдөөнийг  өрнүүлэх  зарчмыг  
баримтлах ёстой бөгөөд худалдан авах барааны шинж чанар, материал, үйл  
ажиллагааны  шаардагдах  стандартын  талаар  тодорхой  тусгана.  Эдгээрийг  
хангасан  нөхцөлд  худалдан  авах  ажиллагааны  хэмнэлттэй,  үр  ашигтай,  
шударга  байх  зорилго  хэрэгжиж,  тендерүүдийн  шаардлагад  нийцэх  байдал  
баталгаажиж, тендерийн үнэлгээ зөв хийгдэнэ. 
Техникийн тодорхойлолт боловсруулахдаа батлагдсан төсөвт багтаан эрчим  
хүч,  байгалийн  нөөцийн  хэмнэлттэй,  эдийн  засгийн  үр  ашигтай  хэрэглээний  
тогтвортой байдлыг хангах, байгаль орчин, хүний эрүүл мэндэд сөрөг нөлөөгүй,  
хүлэмжийн  хийн  ялгарал  болон  хаягдал  багатай,  уур  амьсгалын  өөрчлөлтөд 

Returned: 2
тохиолдолд  Захиалагчийн  шаардсан  техникийн  чадавх,  
туршлагын шаардлагыг хангасан байна.
2.4.

In [11]:
print(docs[5].page_content)

2.1- Байгууллагын дансны үлдэгдэл /Харилцагч банкны/
2.2-  Арилжааны  банкны  зээлийн  авч  болох  мөнгөн  дүн  бүхий  
зээлийн  Тодорхойлолт  (Банкны  судлан  үзэж  зээл  олгох  
боломжтой гэсэн мэдэгдлийг үнэлгээнд харгалзахгүй). 
2.3-Үнэт  цаас,  Засгийн  газрын  бонд  болон  Үнэт  цаасны  зах  
зээлийн тухай хуулийн 5 дугаар зүйлийн 5.1 дэх хэсэгт заасан  
үнэт цааснууд. 
2.4-  Түргэн  хөрвөх  чадвартай  хөрөнгө  болон  авах  боломжтой  
зээлийн  нийт  дүнг  Сангийн  сайдын  2015  оны  343  дугаар  
тушаалаар батлагдсан “Санхүүгийн тайланд шинжилгээ хийх  
үлгэрчилсэн загвар”-ын 3.2.2 /е/-д “Түргэн хөрвөх чадвартай  
хөрөнгө=Мөнгө  Түүнтэй  Адилтгах  Хөрөнгө  +  Санхүүгийн  
хөрөнгө + Дансны авлага гэсэн үзүүлэлтүүдээр дүгнэнэ” гэж  
заасны дагуу тодорхойлно


In [12]:
prompt = 'ижил төстэй ямар бараанууд байгаа вэ'
docs = get_and_print_docs(prompt, vector_db = 'chroma', k = 10)

Prompt: ижил төстэй ямар бараанууд байгаа вэ
Vector DB: chroma

Returned: 1
2.Оролцогч,  түүний  түншлэлийн  гишүүн  худалдан  авах  ажиллагаанд  оролцох  
холбоотой хуулиар хориглосон нөхцөлд хамаарахгүй бөгөөд үүний дотор дараах  
нөхцөлийг гэрчилж байна:
2.1.  Төрийн  болон  орон  нутгийн  өмчийн  хөрөнгөөр  бараа,  ажил,  үйлчилгээ  
худалдан авах тухай хуулийн 7 дугаар зүйлийн 7.1-д заасан нөхцөл үүсээгүй;
2.1.Нөлөө бүхий хувьцаа эзэмшигч, түүний эцсийн өмчлөгч, хөрөнгө эзэмшигч,  
хамтрагч, эсхүл эдгээрийг гүйцэтгэж байгаа хувиараа аж ахуй эрхлэгч нь  
Нийтийн албанд нийтийн болон хувийн ашиг сонирхлыг зохицуулах, ашиг  
сонирхлын  зөрчлөөс  урьдчилан  сэргийлэх  тухай  хуулийн  20  дугаар  
зүйлийн  20.2  дахь  хэсэгт  заасан  албан  тушаал  эрхэлж  байсан  болон  
эрхэлж байгаа хүн, эсхүл түүнтэй хамаарал бүхий этгээд биш; 

Returned: 2
дараах засвар, үйлчилгээ21.1.Санал болгож буй барааны нийлүүлэлтийн  
дараах  засвар,  үйлчилгээтэй  холбоотой  өгөгдлийн  
хүснэгтэд заасан бо

In [13]:
prompt = 'ижил төстэй ямар барааны гэрээнд юу байгаа вэ'
docs = get_and_print_docs(prompt, vector_db = 'chroma', k = 10)

Prompt: ижил төстэй ямар барааны гэрээнд юу байгаа вэ
Vector DB: chroma

Returned: 1
Энэ гэрээ нь захиалагчаас бараа, түүнтэй холбогдох бараа худалдан авах харилцааг  
зохицуулсан  жишиг  заалтуудыг  агуулна.  Гэрээний  нөхцөлийг  боловсруулахад  ГЕН-д  
өөрчлөлт оруулахгүй.
ГЕН-ийн холбогдох зүйл, заалтыг тодруулах, түүнд нэмэлт, өөрчлөлт оруулах бол ГТН-ийн  
холбогдох заалтад тухайн бараа худалдан авах нөхцөлд тохируулан өөрчлөлт оруулан  
ашиглана. Гэрээний нөхцөлийг бэлтгэхэд дараахыг анхаарна: 
1.ГЕН-ийн  аль  нэг  зүйлд  тодорхой  мэдээллийг  ГТН-д  тусгахаар  заасан  бол  
холбогдох  мэдээллийг  ГТН-д  заавал  оруулна.  ГЕН-ийн  аль  нэг  зүйл,  заалтыг  
тухайн бараа худалдан авахад хэрэглэхгүй бол энэ тухай ГТН-д тусгах;
2.Гэрээ баталгаажуулах маягтад шалгарсан оролцогчийн мэдээлэл болон “хамгийн  
сайн” үнэлэгдсэн тендерийн мэдээллийг тусгах; 

Returned: 2
1.1.5.“Талууд” гэж Захиалагч болон Нийлүүлэгчийг  
хамтад нэрлэснийг;
1.1.6.“Хууль”  гэж  Төрийн  болон  орон  нутгийн  

In [15]:
# Returns info about bunch of маяг
prompt = 'ижил төстэй гэрээ?'
docs = get_and_print_docs(prompt, vector_db = 'chroma', k = 10)

Prompt: ижил төстэй гэрээ?
Vector DB: chroma

Returned: 1
2.Оролцогч,  түүний  түншлэлийн  гишүүн  худалдан  авах  ажиллагаанд  оролцох  
холбоотой хуулиар хориглосон нөхцөлд хамаарахгүй бөгөөд үүний дотор дараах  
нөхцөлийг гэрчилж байна:
2.1.  Төрийн  болон  орон  нутгийн  өмчийн  хөрөнгөөр  бараа,  ажил,  үйлчилгээ  
худалдан авах тухай хуулийн 7 дугаар зүйлийн 7.1-д заасан нөхцөл үүсээгүй;
2.1.Нөлөө бүхий хувьцаа эзэмшигч, түүний эцсийн өмчлөгч, хөрөнгө эзэмшигч,  
хамтрагч, эсхүл эдгээрийг гүйцэтгэж байгаа хувиараа аж ахуй эрхлэгч нь  
Нийтийн албанд нийтийн болон хувийн ашиг сонирхлыг зохицуулах, ашиг  
сонирхлын  зөрчлөөс  урьдчилан  сэргийлэх  тухай  хуулийн  20  дугаар  
зүйлийн  20.2  дахь  хэсэгт  заасан  албан  тушаал  эрхэлж  байсан  болон  
эрхэлж байгаа хүн, эсхүл түүнтэй хамаарал бүхий этгээд биш; 

Returned: 2
Маягт 1: Тендер илгээх маягт
[огноо]
[Захиалагч байгууллагын нэр]
Манай  [оролцогчийн нэр]  нь танай байгууллагаас зарласан  [тендер шалгаруулалтын  
нэр, дугаар

# FAISS

In [16]:
prompt = 'ижил төстэй ямар бараанууд байгаа вэ'
docs = get_and_print_docs(prompt, vector_db = 'faiss', k = 10)

Prompt: ижил төстэй ямар бараанууд байгаа вэ
Vector DB: faiss

Returned: 1
2.Оролцогч,  түүний  түншлэлийн  гишүүн  худалдан  авах  ажиллагаанд  оролцох  
холбоотой хуулиар хориглосон нөхцөлд хамаарахгүй бөгөөд үүний дотор дараах  
нөхцөлийг гэрчилж байна:
2.1.  Төрийн  болон  орон  нутгийн  өмчийн  хөрөнгөөр  бараа,  ажил,  үйлчилгээ  
худалдан авах тухай хуулийн 7 дугаар зүйлийн 7.1-д заасан нөхцөл үүсээгүй;
2.1.Нөлөө бүхий хувьцаа эзэмшигч, түүний эцсийн өмчлөгч, хөрөнгө эзэмшигч,  
хамтрагч, эсхүл эдгээрийг гүйцэтгэж байгаа хувиараа аж ахуй эрхлэгч нь  
Нийтийн албанд нийтийн болон хувийн ашиг сонирхлыг зохицуулах, ашиг  
сонирхлын  зөрчлөөс  урьдчилан  сэргийлэх  тухай  хуулийн  20  дугаар  
зүйлийн  20.2  дахь  хэсэгт  заасан  албан  тушаал  эрхэлж  байсан  болон  
эрхэлж байгаа хүн, эсхүл түүнтэй хамаарал бүхий этгээд биш; 

Returned: 2
дараах засвар, үйлчилгээ21.1.Санал болгож буй барааны нийлүүлэлтийн  
дараах  засвар,  үйлчилгээтэй  холбоотой  өгөгдлийн  
хүснэгтэд заасан бол

In [17]:
# Not bad?
prompt = 'техникийн ямар туршлага шаардах вэ'
docs = get_and_print_docs(prompt, vector_db = 'faiss', k = 10)

Prompt: техникийн ямар туршлага шаардах вэ
Vector DB: faiss

Returned: 1
шаардлага19.1.Хуулийн  17.4-т  зааснаар  оролцогч  өгөгдлийн  
хүснэгтэд  заасан  техникийн  чадавх  болон  туршлагын  
шаардлагыг хангасан байна.
19.2.Техникийн  болон  туршлагын  чадавхын  
шаардлагыг хангах нотлох баримт ирүүлэхийг шаардах  
бол  хуулийн  17.2-т  заасан  баримт  бичгээс  сонгож  
өгөгдлийн хүснэгтэд заана.
19.3.Бараа  нийлүүлэх,  дагалдах  үйлчилгээ  үзүүлэхэд  
шаардлагатай  зөвшөөрөл,  эсхүл  гэрчилгээ  байгаа  бол  
өгөгдлийн хүснэгтэд заана. 

Returned: 2
тохиолдолд  Захиалагчийн  шаардсан  техникийн  чадавх,  
туршлагын шаардлагыг хангасан байна.
2.4.Нийлүүлэгч  нь  Барааг  нийлүүлэхэд  зайлшгүй  
шаардлагатай  баримт  бичиг,  бусад  зүйлийг  гаргуулахыг  
хүссэн тохиолдолд Захиалагч ердийн боломжит хугацаанд  
гаргаж өгнө.
Бараа нийлүүлэх  газар 2.5.Бараа нийлүүлэх болон Дагалдах үйлчилгээ үзүүлэх  
газрыг ГТН-д заана.
Бараа  нийлүүлэх  
хугацаа, түүнийг сунгах2.6.Бараа нийлүүлэх хугацааг

In [18]:
print(docs[0].page_content)

шаардлага19.1.Хуулийн  17.4-т  зааснаар  оролцогч  өгөгдлийн  
хүснэгтэд  заасан  техникийн  чадавх  болон  туршлагын  
шаардлагыг хангасан байна.
19.2.Техникийн  болон  туршлагын  чадавхын  
шаардлагыг хангах нотлох баримт ирүүлэхийг шаардах  
бол  хуулийн  17.2-т  заасан  баримт  бичгээс  сонгож  
өгөгдлийн хүснэгтэд заана.
19.3.Бараа  нийлүүлэх,  дагалдах  үйлчилгээ  үзүүлэхэд  
шаардлагатай  зөвшөөрөл,  эсхүл  гэрчилгээ  байгаа  бол  
өгөгдлийн хүснэгтэд заана.


In [19]:
print(docs[3].page_content)

18, 19-д чадавхын болон туршлагын шаардлагын 40-аас  
дээш хувийг, түншлэлийн гишүүн тус бүр 25 ба түүнээс  
дээш хувийг хангасан байна.
18.Санхүүгийн 
чадавхын шаардлага18.1.Хуулийн  16.5-д  зааснаар  оролцогч  
өгөгдлийн  хүснэгтэд  заасан  санхүүгийн  чадавхын  
шаардлага хангасан байна.
18.2.Санхүүгийн  чадавхын  шаардлагыг  хангах  нотлох  
баримт  ирүүлэхийг  шаардах  бол  хуулийн  16.2,  16.4-т  
заасан баримт бичгээс сонгож өгөгдлийн хүснэгтэд заана.
18.3.Оролцогчийн дансны мэдээлэл, түүний үлдэгдлийн  
талаар  харилцагч  банкнаас  мэдэгдсэнээс  бусад  банк,  
санхүүгийн  байгууллагын  мэдэгдлийг  оролцогчийн  
санхүүгийн  чадавхыг  хянан  үзэхэд  харгалзан  үзэхгүй.  
Захиалагч  энд  зааснаас  бусад  асуудлаар  харилцагч  
банкны тодорхойлолт ирүүлэхийг шаардахгүй.


In [20]:
prompt = 'ижил төстэй гэрээний туршлага юу юу тооцох вэ'
docs = get_and_print_docs(prompt, vector_db = 'faiss', k = 10)

Prompt: ижил төстэй гэрээний туршлага юу юу тооцох вэ
Vector DB: faiss

Returned: 1
нийлүүлсэн гэрээг тооцно.
1.Бараа нийлүүлсэн гэрээний эх хувийг цахим тендерт хавсаргаж  
ирүүлэх  ба  хуулбар  хувь  бол  нотариатаар  баталгаажуулсан  
байна, 
2.Гэрээ дүгнэсэн  акт эх хувийг цахим тендерт хавсаргаж ирүүлэх  
ба хуулбар хувь бол нотариатаар баталгаажуулсан байна,
3.Тухай гэрээний гэрээний захиалагч байгууллагын тодорхойлолтыг  
хавсаргаж  ирүүлнэ.  Захиалагч  байгууллагын  тодорхойлолтод  
чанарын  шаардлага  хангаагүй,  хугацаа  хэтэрсэн  зэрэг  
тодорхойлолт ирүүлсэн тохиолдолд тухайн тендерээс татгалзах  
үндэслэл болно.
/Ижил  төстэй  гэрээний  туршлагад  дээрх  баримт  бичгийн  
бүрдлийн аль нэг ирүүлээгүй тохиолдолд тендерээс татгалзах  
үндэслэл болно./
ТШЗ 19.2Техникийн  чадавх  болон  туршлагын  шаардлагыг  нотлох  баримт  
бичиг:Шаардана. 

Returned: 2
Энэ гэрээ нь захиалагчаас бараа, түүнтэй холбогдох бараа худалдан авах харилцааг  
зохицуулсан  жишиг  заалтуудыг  агуулна. 

In [21]:
prompt = 'ижил төстэй гэрээний туршлагад ямар бичиг баримт шаардах вэ'
docs = get_and_print_docs(prompt, vector_db = 'faiss', k = 10)

Prompt: ижил төстэй гэрээний туршлагад ямар бичиг баримт шаардах вэ
Vector DB: faiss

Returned: 1
нийлүүлсэн гэрээг тооцно.
1.Бараа нийлүүлсэн гэрээний эх хувийг цахим тендерт хавсаргаж  
ирүүлэх  ба  хуулбар  хувь  бол  нотариатаар  баталгаажуулсан  
байна, 
2.Гэрээ дүгнэсэн  акт эх хувийг цахим тендерт хавсаргаж ирүүлэх  
ба хуулбар хувь бол нотариатаар баталгаажуулсан байна,
3.Тухай гэрээний гэрээний захиалагч байгууллагын тодорхойлолтыг  
хавсаргаж  ирүүлнэ.  Захиалагч  байгууллагын  тодорхойлолтод  
чанарын  шаардлага  хангаагүй,  хугацаа  хэтэрсэн  зэрэг  
тодорхойлолт ирүүлсэн тохиолдолд тухайн тендерээс татгалзах  
үндэслэл болно.
/Ижил  төстэй  гэрээний  туршлагад  дээрх  баримт  бичгийн  
бүрдлийн аль нэг ирүүлээгүй тохиолдолд тендерээс татгалзах  
үндэслэл болно./
ТШЗ 19.2Техникийн  чадавх  болон  туршлагын  шаардлагыг  нотлох  баримт  
бичиг:Шаардана. 

Returned: 2
Энэ гэрээ нь захиалагчаас бараа, түүнтэй холбогдох бараа худалдан авах харилцааг  
зохицуулсан  жишиг  заалтуу

In [23]:
prompt = 'ижил төстэй гэрээ, үнийн дүн, бараа нийлүүлэх'
docs = get_and_print_docs(prompt, vector_db = 'faiss', k = 10)

Prompt: ижил төстэй гэрээ, үнийн дүн, бараа нийлүүлэх
Vector DB: faiss

Returned: 1
тухайн бараа худалдан авахад хэрэглэхгүй бол энэ тухай ГТН-д тусгах;
2.Гэрээ баталгаажуулах маягтад шалгарсан оролцогчийн мэдээлэл болон “хамгийн  
сайн” үнэлэгдсэн тендерийн мэдээллийг тусгах;
3.Гэрээний  үнийг  тендерийн  үнэд  залруулга  болон  хөнгөлөлтийг  тооцож,  нэг  
валютад хөрвүүлсний дараах үнээр тогтоох; 
4.Нийлүүлэх бараа, дагалдах үйлчилгээний дэлгэрэнгүй тодорхойлолт, зураг зэргийг  
гэрээнд хавсаргах;
5.ГЕН, ГТН болон холбогдох хавсралтууд нь өөр хоорондоо зөрчилдөхгүй байх;  
6.Гэрээ  баталгаажуулах  маягтын  хаалтан  дотор  бичсэн  жишээ,  зааврыг  тендер  
шалгаруулалтын үр дүнд шалгарсан тендерийн нөхцөлд тохирох үг өгүүлбэр, тоо  
хэмжээ зэрэгт нийцүүлэн тусгана. ГТН-д заасан зарим жишээ, нөхцөлийг зөвхөн 

Returned: 2
1.1.5.“Талууд” гэж Захиалагч болон Нийлүүлэгчийг  
хамтад нэрлэснийг;
1.1.6.“Хууль”  гэж  Төрийн  болон  орон  нутгийн  
өмчийн  хөрөнгөөр  бараа,  ажил,  үйлчилгээ 